In [1]:
import os, sys
import pandas as pd
import re

#import the functions
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import rtcw_functions as RTCW
reload(RTCW)

db_name = 'rtcw_db.db'
demos_folder = 'demos'
cwd = os.getcwd()
demos_path = os.path.join(cwd, '../', demos_folder)

exe_name = 'Anders.Gaming.LibTech3.exe'
root_path = 'C:\\Users\\Jelle\\Documents\\GIT\\rtcw_demo_analyzer' #for executable we need a direct path

In [2]:
#get demos_dct
demos_dct = RTCW.make_dictionary(demos_path)

parsed 100 matches
parsed 200 matches
parsed 300 matches
parsed 400 matches
finished parsing all matches!


In [3]:
#load obituaries in a df
import sqlite3_connector
reload(sqlite3_connector)
db = sqlite3_connector.sqlite3_connector(os.path.join(root_path, db_name))

In [25]:
obituary_df = db.get_table_as_df(table_name= 'obituary')
demo_df = db.get_table_as_df(table_name= 'demo')
player_df = db.get_table_as_df(table_name= 'player')
chatmessages_df = db.get_table_as_df(table_name= 'chatmessage')
chatmessages_df = RTCW.add_match_data(chatmessages_df, player_df, demos_dct, what_df = 'chatmessages_df')
chatmessages_df = RTCW.feature_extraction_chat(chatmessages_df)

obituary table fetched in 0.44 seconds
demo table fetched in 0.01 seconds
player table fetched in 0.05 seconds
chatmessage table fetched in 0.64 seconds


In [29]:
docs_df = chatmessages_df.loc[chatmessages_df['DocsAll'] | chatmessages_df['TimelimitHit']]

In [64]:
def get_docruns(chatmessages_df, min_docrun_length, max_timeleft, docs_succesful = None, verbose = True):
    
    #empty lists that we will populate and use to make final df
    pd_md5 = []
    pd_start_docrun_dwtime = []
    pd_end_docrun_dwtime = []
    pd_start_docrun_secsleft = []
    pd_end_docrun_secsleft = []
    pd_docrun_duration = []
    pd_times_docs_lost = []
    pd_won_round = []
    pd_demo_name = []
    pd_match_name = []
    
    #helper variables for verbose
    counter = 0
    total_demos = obituary_df.szMd5.nunique()
    
    for demo in chatmessages_df.szMd5.unique():
        #evt hier nog cutten op timelimit hit of docsevents zoals cell hierboven voor speed
        df_demo = chatmessages_df.loc[(chatmessages_df['szMd5'] == demo) & (chatmessages_df['InMatch'])]
        demo_name = df_demo.demoName.unique()
        match_name = df_demo.matchName.unique()
        counter += 1
        
        arr = df_demo.as_matrix(columns = ['DocsEvents', 'TimelimitHit', 'dwTime', 'SecondsLeftInRound'])
        taken_docs_bool = False
        docs_lost_times = 0
        
        for row in range(len(arr)):
            chat = arr[row, :]
            
            #taken docs
            if chat[0] == 1:
                if taken_docs_bool == False: 
                    start_dwtime = chat[2]
                    start_timeleft = chat[3]
                    taken_docs_bool = True
                else:
                    docs_lost_times += 1

            #transmitted docs
            if chat[0] == 2:
                end_dwtime = chat[2]
                end_timeleft = chat[3]
                taken_docs_bool == False
                docs_succes = True
                
                pd_md5.append(demo)
                pd_start_docrun_dwtime.append(start_dwtime)
                pd_end_docrun_dwtime.append(end_dwtime)
                pd_start_docrun_secsleft.append(start_timeleft)
                pd_end_docrun_secsleft.append(end_timeleft)
                pd_docrun_duration.append(start_timeleft - end_timeleft)
                pd_times_docs_lost.append(docs_lost_times)
                pd_won_round.append(docs_succes)
                pd_demo_name.append(demo_name[0])
                pd_match_name.append(match_name[0])
            
                docs_lost_times = 0
                
            #returned docs / timelimit hit
            if (chat[0] == 0 or chat[1] == 1):
                end_dwtime = chat[2]
                end_timeleft = chat[3]
                taken_docs_bool == False
                docs_succes = False
                
                pd_md5.append(demo)
                pd_start_docrun_dwtime.append(start_dwtime)
                pd_end_docrun_dwtime.append(end_dwtime)
                pd_start_docrun_secsleft.append(start_timeleft)
                pd_end_docrun_secsleft.append(end_timeleft)
                pd_docrun_duration.append(start_timeleft - end_timeleft)
                pd_times_docs_lost.append(docs_lost_times)
                pd_won_round.append(docs_succes)
                pd_demo_name.append(demo_name[0])
                pd_match_name.append(match_name[0])
                
                docs_lost_times = 0
                
            #if demo stopped before end of docrun
            
            
        #verbose shizzle
        if counter % 100 == 0:
            if verbose:
                print 'scanned ' + str(counter) + ' demos of ' + str(total_demos) + ' demos in total' 
    
    print 'all done!'
    
    
    #make final dataframe where 1 row is a spree with all the necessary info
    df_docs = pd.DataFrame(
    {'md5': pd_md5,
     'start_dwtime': pd_start_docrun_dwtime,
     'end_dwtime': pd_end_docrun_dwtime,
     'start_secsleft': pd_start_docrun_secsleft,
     'end_secsleft': pd_end_docrun_secsleft,
     'duration': pd_docrun_duration,
     'times_lost_docs': pd_times_docs_lost,
     'won_round': pd_won_round,
     'demo': pd_demo_name,
     'match': pd_match_name,
    })
    
    return df_docs    

In [65]:
chattest = chatmessages_df.loc[chatmessages_df['szMd5'] == '2c60f8f66f42642e8e3c12d3e45d4500']

In [66]:
res = get_docruns(chattest, '1', '1', None, verbose = True)
res[['start_dwtime', 'end_dwtime', 'start_secsleft', 'end_secsleft', 'duration', 'times_lost_docs', 'won_round']]

all done!


start_dwtime  end_dwtime  start_secsleft  end_secsleft  duration  \
0       43860900    43902600             495           453        42   
1       43860900    44109150             495           246       249   
2       43860900    44228900             495           127       368   
3       43860900    44600050             495           190       305   
4       43860900    44655200             495           134       361   
5       43860900    44767500             495            22       473   
6       43860900    44790050             495             0       495   
7       43860900    45263600             495           356       139   
8       43860900    45619700             495             0       495   
9       43860900    45846150             495           428        67   
10      43860900    45892500             495           381       114   
11      43860900    46204050             495           373       122   
12      43860900    46353800             495           223       272   
13      43860900    46448150             495           129       366   
14      43860900    46685750             495           288       207   
15      43860900    46738500             495           235       260   

    times_lost_docs  won_round  
0                 0      False  
1                 1      False  
2                 1       True  
3                 1      False  
4                 1      False  
5                 1      False  
6                 0      False  
7                 1      False  
8                 1      False  
9                 1      False  
10                1       True  
11                1      False  
12                1      False  
13                3       True  
14                2      False  
15                1       True

In [49]:
chatmessages_df.loc[(chatmessages_df['DocsEvents'] != -1) & (chatmessages_df['szMd5'] == '2c60f8f66f42642e8e3c12d3e45d4500')]

dwSeq                             szMd5  \
424067  419762  2c60f8f66f42642e8e3c12d3e45d4500   
424069  419764  2c60f8f66f42642e8e3c12d3e45d4500   
424092  419787  2c60f8f66f42642e8e3c12d3e45d4500   
424094  419789  2c60f8f66f42642e8e3c12d3e45d4500   
424103  419798  2c60f8f66f42642e8e3c12d3e45d4500   
424106  419801  2c60f8f66f42642e8e3c12d3e45d4500   
424155  419850  2c60f8f66f42642e8e3c12d3e45d4500   
424157  419852  2c60f8f66f42642e8e3c12d3e45d4500   
424158  419853  2c60f8f66f42642e8e3c12d3e45d4500   
424160  419855  2c60f8f66f42642e8e3c12d3e45d4500   
424174  419869  2c60f8f66f42642e8e3c12d3e45d4500   
424176  419871  2c60f8f66f42642e8e3c12d3e45d4500   
424345  420040  2c60f8f66f42642e8e3c12d3e45d4500   
424350  420045  2c60f8f66f42642e8e3c12d3e45d4500   
424438  420133  2c60f8f66f42642e8e3c12d3e45d4500   
424502  420197  2c60f8f66f42642e8e3c12d3e45d4500   
424505  420200  2c60f8f66f42642e8e3c12d3e45d4500   
424507  420202  2c60f8f66f42642e8e3c12d3e45d4500   
424514  420209  2c60f8f66f42642e8e3c12d3e45d4500   
424606  420301  2c60f8f66f42642e8e3c12d3e45d4500   
424611  420306  2c60f8f66f42642e8e3c12d3e45d4500   
424672  420367  2c60f8f66f42642e8e3c12d3e45d4500   
424673  420368  2c60f8f66f42642e8e3c12d3e45d4500   
424676  420371  2c60f8f66f42642e8e3c12d3e45d4500   
424679  420374  2c60f8f66f42642e8e3c12d3e45d4500   
424684  420379  2c60f8f66f42642e8e3c12d3e45d4500   
424702  420397  2c60f8f66f42642e8e3c12d3e45d4500   
424815  420510  2c60f8f66f42642e8e3c12d3e45d4500   
424820  420515  2c60f8f66f42642e8e3c12d3e45d4500   
424825  420520  2c60f8f66f42642e8e3c12d3e45d4500   
424830  420525  2c60f8f66f42642e8e3c12d3e45d4500   
424852  420547  2c60f8f66f42642e8e3c12d3e45d4500   

                                     szMessage  bPlayer    dwTime  \
424067      Axis have stolen Allied Documents!       -1  43860900   
424069  Allies have returned Allied Documents!       -1  43902600   
424092      Axis have stolen Allied Documents!       -1  44104350   
424094  Allies have returned Allied Documents!       -1  44109150   
424103      Axis have stolen Allied Documents!       -1  44173850   
424106         Axis transmitted the documents!       -1  44228900   
424155      Axis have stolen Allied Documents!       -1  44568700   
424157     Allies have returned the objective!       -1  44600050   
424158      Axis have stolen Allied Documents!       -1  44610200   
424160     Allies have returned the objective!       -1  44655200   
424174      Axis have stolen Allied Documents!       -1  44762150   
424176  Allies have returned Allied Documents!       -1  44767500   
424345      Axis have stolen Allied Documents!       -1  45248100   
424350  Allies have returned Allied Documents!       -1  45263600   
424438      Axis have stolen Allied Documents!       -1  45610700   
424502      Axis have stolen Allied Documents!       -1  45838050   
424505  Allies have returned Allied Documents!       -1  45846150   
424507      Axis have stolen Allied Documents!       -1  45847750   
424514         Axis transmitted the documents!       -1  45892500   
424606      Axis have stolen Allied Documents!       -1  46183450   
424611  Allies have returned Allied Documents!       -1  46204050   
424672      Axis have stolen Allied Documents!       -1  46352750   
424673  Allies have returned Allied Documents!       -1  46353800   
424676      Axis have stolen Allied Documents!       -1  46362500   
424679      Axis have stolen Allied Documents!       -1  46372600   
424684      Axis have stolen Allied Documents!       -1  46383250   
424702         Axis transmitted the documents!       -1  46448150   
424815      Axis have stolen Allied Documents!       -1  46670700   
424820      Axis have stolen Allied Documents!       -1  46678400   
424825  Allies have returned Allied Documents!       -1  46685750   
424830      Axis have stolen Allied Documents!       -1  46698800   
424852         Axis transmitted the documents!       -1  46738500   

       szTimeS

In [ ]:
from chathelper import DocsStolen
from chathelper import DocsReturned
from chathelper import DocsTransmitted
from chathelper import DocsAll
from chathelper import DynamitePlanted
from chathelper import DynamiteDefused
from chathelper import DynamiteExploded
from chathelper import DynamiteAll

In [ ]:
chatmessages_df.as_matrix(columns = ['DocsEvents', 'TimelimitHit', 'dwTime'])